In [1]:
!pip install transformers

In [2]:
!pip install datasets

In [3]:
from pathlib import Path
import os
import warnings
import random

import re
import json
import pandas as pd
from datasets import ClassLabel, Dataset, DatasetDict
import librosa
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.display import Audio

from IPython.display import display, HTML


import torch
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = "/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2"

In [6]:
#!cd "/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2"
#!cd path
!find "/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2" -type f | wc -l

10184


In [7]:
with open("/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/text") as file_name:
    data = file_name.read().split("\n")

In [8]:
file_name=[]
sentences=[]

In [9]:
print(data)

['SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part10\trais wa tanzania jakaya mrisho kikwete', 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part100\tyanayo andaliwa nami pendo pondo idhaa ya kiswahili', 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part101\tinayokutangazia moja kwa moja kutoka jijini dar es salaam tanzania', 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part102\tjuma hili bara la afrika limeshuhudia raia wa nchi za niger', 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part103\twakipiga kura ya maoni ilikufanya mabadiliko ya', 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part104\tkule abidjan raia wa jiji hilo', 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part105\twalipata fursa ya kutumia haki yao ya msingi', 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part106\twaziri mkuu wa zamani alasane watara', 'SWH-05-20101106_16k-

In [10]:
for i in data:
    file_split= i.split('\t')
    if len(file_split) ==2:
      file_name.append(file_split[0])
      sentences.append(file_split[1])
    else:
      file_name.append(file_split[0])
      sentences.append("")  # some files did not have sentences.


In [11]:
audio_data = {
    'ID':file_name,
    'transcription':sentences
}

In [12]:
print(len(audio_data['ID']))
print(len(audio_data['transcription']))

10181
10181


In [13]:
swahili_train = pd.DataFrame(audio_data)
swahili_train = swahili_train[:10000]
swahili_train

,ID,transcription
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya
...,...,...
9995,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,swala ulinzi litaanza kutekelezwa na majeshi
9996,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,uliopendekezwa na rais ali abdulahi salle
9997,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,ya nchi hiyo katika majimbo manne na miji mita...
9998,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,kiongozi huyo ambaye yupo karibu na rais hamid...


In [14]:
# Create dict of ID's and file paths
paths = {}
#path = '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/train/'
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".wav"):
            audio_path = os.path.join(root, file)
            p = Path(audio_path)
            id = p.parts[-1].split('.')[0]
            paths[id] = audio_path

In [15]:
print(paths)

{'SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part98': '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-15-20101104/SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part98.wav', 'SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part99': '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-15-20101104/SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part99.wav', 'SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part97': '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-15-20101104/SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part97.wav', 'SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part84': '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-15-20101104/SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part84.wav', 'SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part86': '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SW

In [16]:
swahili_train.iloc[2]

ID               SWH-05-20101106_16k-emission_swahili_05h30_-_0...
transcription    inayokutangazia moja kwa moja kutoka jijini da...
Name: 2, dtype: object

In [17]:
swahili_train.shape

(10000, 2)

In [18]:
Audio(paths[swahili_train.iloc[1]["ID"].strip()])

In [19]:
swahili_train['file'] = [paths[id.strip()] for id in list(swahili_train['ID'])]

In [20]:
swahili_train

,ID,transcription,file
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
...,...,...,...
9995,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,swala ulinzi litaanza kutekelezwa na majeshi,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
9996,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,uliopendekezwa na rais ali abdulahi salle,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
9997,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,ya nchi hiyo katika majimbo manne na miji mita...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
9998,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,kiongozi huyo ambaye yupo karibu na rais hamid...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...


In [21]:
swahili_train = swahili_train.drop(["ID"], axis = 1)

In [22]:
swahili_train

,transcription,file
0,rais wa tanzania jakaya mrisho kikwete,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
1,yanayo andaliwa nami pendo pondo idhaa ya kisw...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
2,inayokutangazia moja kwa moja kutoka jijini da...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
3,juma hili bara la afrika limeshuhudia raia wa ...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
4,wakipiga kura ya maoni ilikufanya mabadiliko ya,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
...,...,...
9995,swala ulinzi litaanza kutekelezwa na majeshi,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
9996,uliopendekezwa na rais ali abdulahi salle,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
9997,ya nchi hiyo katika majimbo manne na miji mita...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...
9998,kiongozi huyo ambaye yupo karibu na rais hamid...,/content/drive/MyDrive/Colab Notebooks/DeepL2/...


In [23]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [24]:
swahili = DatasetDict({'train': Dataset.from_pandas(swahili_train[0:2000]), 'test': Dataset.from_pandas(swahili_train[2000:2400])})

In [25]:
chars_to_ignore_regex = '[\"\?\.\!\-\;\:\(\)\,]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [26]:
swahili = swahili.map(remove_special_characters)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [27]:
swahili

DatasetDict({
    train: Dataset({
        features: ['transcription', 'file'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['transcription', 'file'],
        num_rows: 400
    })
})

In [28]:
show_random_elements(swahili['train'], num_examples=10)

,transcription,file
0,msaada wa marekani kutoa dola milioni mia moja na hamsini,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101111/SWH-05-20101111_emission_swahili_05h30_-_06h00_tu_20101111_part51.wav
1,la usalama katika baraza la umoja wa mataifa mwandishi wetu zuhra mwera,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101109/SWH-05-20101109_emission_swahili_05h30_-_06h00_tu_20101109_part10.wav
2,ina ina nguvu sana either nikiuchumi kitechnologia,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101110/SWH-05-20101110_16k-emission_swahili_15h00_-_16h00_tu_20101110_part455.wav
3,msemaji wa polisi hussen kanali felex kureithi j,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101113/SWH-05-20101113_16k-emission_swahili_05h30_-_06h00_tu_20101113_part107.wav
4,na hali kadhalika shule kufungwa pamoja na huduma zingine katika sekta za afya,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101111/SWH-05-20101111_emission_swahili_05h30_-_06h00_tu_20101111_part95.wav
5,imesema ipo tayari kutoa ushirikiano,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101113/SWH-05-20101113_16k-emission_swahili_05h30_-_06h00_tu_20101113_part37.wav
6,kwa muda mrefu sasa kanisa hilo limekuwa likipinga utumiaji wa mipira kondom,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101121/SWH-05-20101121_16k-emission_swahili_05h30_-_06h00_tu_20101121_part9.wav
7,hilda brian bisho ni mwanaharakati na hapa anatoa mtazamo wake,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101113/SWH-05-20101113_16k-emission_swahili_05h30_-_06h00_tu_20101113_part64.wav
8,aa kuja kushuhudia sherehe hii ya kipekee ya kumwapisha,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part19.wav
9,kwa sheria ya wafaransa bunge na senate,/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec2/wav/SWH-05-20101110/SWH-05-20101110_16k-emission_swahili_15h00_-_16h00_tu_20101110_part12.wav


In [29]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [30]:
vocabs = swahili.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=swahili.column_names["train"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [31]:
vocab_list = list(set(vocabs["train"]["vocab"][0]))

In [32]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 'l': 1,
 '<': 2,
 'd': 3,
 'i': 4,
 'e': 5,
 'u': 6,
 'o': 7,
 '3': 8,
 '2': 9,
 'a': 10,
 'z': 11,
 'f': 12,
 't': 13,
 'k': 14,
 'y': 15,
 'n': 16,
 'p': 17,
 'c': 18,
 'j': 19,
 '6': 20,
 'b': 21,
 '0': 22,
 's': 23,
 'x': 24,
 'v': 25,
 '_': 26,
 '1': 27,
 'm': 28,
 'q': 29,
 'r': 30,
 'h': 31,
 'w': 32,
 '5': 33,
 '>': 34,
 'g': 35}

In [33]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [34]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

38

In [35]:
# save our vocab to json file
#path = '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec/train/'
with open(path+'vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [36]:
my_vocab = path+'vocab.json'
tokenizer = Wav2Vec2CTCTokenizer(f"{my_vocab}", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [37]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [38]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [39]:
import librosa
import warnings

warnings.filterwarnings("ignore")

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["file"], sr = 16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription"]
    return batch

In [40]:
swahili = swahili.map(speech_file_to_array_fn, remove_columns=swahili.column_names["train"], num_proc=1)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [41]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [42]:
swahili_prepared = swahili.map(prepare_dataset, remove_columns=swahili.column_names["train"], batch_size=32, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [43]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [44]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [45]:
!pip install  jiwer   #install jiwer pour premettre l'utilisation wer (word error rate)

In [46]:
from datasets import load_metric
wer_metric = load_metric("wer")  # Word Error Rate, metric permettant d'evaluer le modele. Une valeur proche de zero montre que le modele performe bien.

In [47]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [48]:
from transformers import Wav2Vec2ForCTC

# load le model pre-entrained
model = Wav2Vec2ForCTC.from_pretrained(
    #"facebook/wav2vec2-xls-r-300m",
    "facebook/wav2vec2-base",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)



Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
#model.freeze_feature_extractor()

In [50]:
torch.cuda.is_available()

True

In [51]:
#!pip install accelerate==0.20.3 -U
!pip install accelerate -U

In [52]:
from transformers import TrainingArguments


In [53]:
import accelerate

In [54]:
accelerate.__version__

'0.23.0'

In [66]:
#path = '/content/drive/MyDrive/Colab Notebooks/DeepL2/wav2vec/train/'
path2= path+'/swahili'
training_args = TrainingArguments(
  output_dir=path2,
  group_by_length=True,
  per_device_train_batch_size=3,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=40000,
  eval_steps=1000,
  logging_steps=1000,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2
)

In [67]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=swahili_prepared["train"],
    eval_dataset=swahili_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [68]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
1000,0.723000,1.152064,0.742857
2000,0.845700,1.201884,0.727964
3000,0.646500,1.038528,0.660182
4000,0.508100,0.930345,0.631003
5000,0.386400,1.083849,0.624012
6000,0.310900,1.000869,0.605471


TrainOutput(global_step=6670, training_loss=0.5385656148061223, metrics={'train_runtime': 1258.3204, 'train_samples_per_second': 15.894, 'train_steps_per_second': 5.301, 'total_flos': 6.427928558627711e+17, 'train_loss': 0.5385656148061223, 'epoch': 10.0})

In [69]:
#save model to hugginface
from huggingface_hub import notebook_login

In [70]:
notebook_login()

In [72]:
!git lfs install

Git LFS initialized.


In [76]:
repo_name = "swahili"

In [77]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/etonkou/swahili/commit/5017567968c4672542b56295a1cc8b6e26407c36', commit_message='Upload tokenizer', commit_description='', oid='5017567968c4672542b56295a1cc8b6e26407c36', pr_url=None, pr_revision=None, pr_num=None)

In [78]:
trainer.push_to_hub()

'https://huggingface.co/etonkou/swahili/tree/main/'